In [1]:
from hyrax import Hyrax
h = Hyrax()
# h.config['data_set']['name'] = 'HyraxRandomDataset'
# h.config['data_set']['random_dataset']['size'] = 10_000
# h.config['data_set']['random_dataset']['shape'] = (3, 41, 41)

[2025-08-18 16:26:09,122 hyrax:INFO] Runtime Config read from: /Users/drew/code/hyrax/src/hyrax/hyrax_default_config.toml


In [2]:
h.config['train']['epochs'] = 1
# h.config['model']['name'] = 'HyraxCNN'
# h.config['model']['hyrax_cnn']['output_classes'] = 3

h.train()

[2025-08-18 16:26:15,390 hyrax.models.model_registry:INFO] Using criterion: torch.nn.CrossEntropyLoss with default arguments.
2025-08-18 16:26:15,405 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset '<hyrax.data_sets.hyr': 
	{'sampler': <hyrax.pytorch_ignite.SubsetSequentialSampler object at 0x16b0263c0>, 'batch_size': 512, 'shuffle': False, 'pin_memory': False}
2025-08-18 16:26:15,406 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset '<hyrax.data_sets.hyr': 
	{'sampler': <hyrax.pytorch_ignite.SubsetSequentialSampler object at 0x16b027740>, 'batch_size': 512, 'shuffle': False, 'pin_memory': False}
/Users/drew/opt/miniconda3/envs/hyrax/lib/python3.12/site-packages/ignite/handlers/tqdm_logger.py:127: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2025/08/18 16:26:15 INFO mlflow.system_metrics

  2%|1         | 1/59 [00:00<?, ?it/s]

[2025-08-18 16:26:21,129 hyrax.pytorch_ignite:INFO] Total training time: 5.60[s]
[2025-08-18 16:26:21,130 hyrax.pytorch_ignite:INFO] Latest checkpoint saved as: /Users/drew/code/hyrax/docs/pre_executed/results/20250818-162611-train-KDT0/checkpoint_epoch_1.pt
[2025-08-18 16:26:21,130 hyrax.pytorch_ignite:INFO] Best metric checkpoint saved as: /Users/drew/code/hyrax/docs/pre_executed/results/20250818-162611-train-KDT0/checkpoint_1_loss=-357.3800.pt
2025/08/18 16:26:21 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/08/18 16:26:21 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
[2025-08-18 16:26:21,145 hyrax.verbs.train:INFO] Finished Training
[2025-08-18 16:26:21,642 hyrax.model_exporters:INFO] Exported model to ONNX format: /Users/drew/code/hyrax/docs/pre_executed/results/20250818-162611-train-KDT0/example_model_opset_20.onnx


HyraxAutoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): GELU(approximate='none')
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): GELU(approximate='none')
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): GELU(approximate='none')
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): GELU(approximate='none')
    (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (9): GELU(approximate='none')
    (10): Flatten(start_dim=1, end_dim=-1)
    (11): Linear(in_features=1024, out_features=64, bias=True)
  )
  (dec_linear): Sequential(
    (0): Linear(in_features=64, out_features=1024, bias=True)
    (1): GELU(approximate='none')
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): GELU(approximate='none')
    (2): 

In [3]:
ds = h.infer()

[2025-08-18 16:26:25,268 hyrax.models.model_registry:INFO] Using criterion: torch.nn.CrossEntropyLoss with default arguments.
[2025-08-18 16:26:25,269 hyrax.verbs.infer:INFO] data set has length 50000
2025-08-18 16:26:25,270 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset '<hyrax.data_sets.hyr': 
	{'sampler': None, 'batch_size': 512, 'shuffle': False, 'pin_memory': False}
[2025-08-18 16:26:25,280 hyrax.verbs.infer:INFO] Saving inference results at: /Users/drew/code/hyrax/docs/pre_executed/results/20250818-162621-infer-zk3O
[2025-08-18 16:26:25,467 hyrax.pytorch_ignite:INFO] Evaluating model on device: mps
[2025-08-18 16:26:25,468 hyrax.pytorch_ignite:INFO] Total epochs: 1


  1%|1         | 1/98 [00:00<?, ?it/s]

[2025-08-18 16:26:30,388 hyrax.pytorch_ignite:INFO] Total evaluation time: 4.92[s]
[2025-08-18 16:26:30,443 hyrax.verbs.infer:INFO] Inference Complete.


In [10]:
ds[[0,1,2,4,5,6]]

tensor([[ 8.2588e-01,  1.8817e+00, -8.6632e-02, -2.6173e-01,  1.6635e+00,
          3.0351e-01,  8.5987e-01,  1.3563e+00,  7.2110e-01, -9.2282e-01,
          1.7628e-01,  9.0702e-01, -5.5977e-01,  7.7089e-01,  5.0296e-01,
         -1.1167e+00,  8.3786e-01, -7.5270e-01,  2.3004e+00,  1.1066e+00,
          8.6355e-01,  1.0416e+00,  2.0365e-01, -1.8310e+00, -1.2457e+00,
         -1.4892e+00, -2.5756e-01, -1.5162e+00, -2.7329e+00, -3.9386e-01,
          1.6330e+00,  4.9924e-03, -5.7766e-02, -1.2319e-01, -6.4349e-02,
         -4.5615e-01, -1.1544e+00, -1.4566e+00,  1.0005e+00, -1.2052e+00,
          1.6361e+00,  2.9691e+00, -6.5868e-01, -8.4884e-01,  8.8773e-01,
          1.0800e-01, -4.1050e-02, -3.5328e+00,  1.9987e+00,  6.2507e-02,
          5.1641e-01,  5.1441e-01,  1.5979e+00,  6.1225e-01, -7.3815e-01,
         -2.8229e+00, -1.1311e+00, -1.7349e+00, -8.8074e-01,  5.8053e-01,
          2.0290e+00,  1.4515e+00, -5.7942e-01, -2.2863e+00],
        [-1.8638e+00,  1.6474e+00,  1.2942e+00,  2

In [ ]:
import numpy as np

tensors = [np.random.rand(3,4,5) for _ in range(512)]
ids = np.arange(512)

In [ ]:
import json
import pyarrow as pa
import pyarrow.parquet as pq

_shape = json.dumps(tensors[0].shape)
flattened_tensors = [tensor.flatten() for tensor in tensors]
table = pa.Table.from_arrays([ids, flattened_tensors], names=['id', 'tensor'])
table = table.replace_schema_metadata({'shape': _shape})

pq.write_table(table, './output.parquet')

In [ ]:
table = pq.read_table('./output.parquet')
retrieved_metadata_bytes = table.schema.metadata.get(b'shape')
reshape_dims = json.loads(retrieved_metadata_bytes)
df = table.to_pandas()
df['tensor'][0].reshape(reshape_dims)

In [ ]:
import pyarrow.parquet as pq

table = pq.read_table('/Users/drew/code/hyrax/docs/pre_executed/results/20250818-160645-infer-Bcjm/output.parquet')

tensor_shape = table.schema.metadata.get(b'tensor_shape')
df = table.to_pandas()

In [ ]:
a = df[0:10]
len(a['tensor'][0])